In [ ]:
from src import collect

df_control = collect.read_feather("/home/ljw/sdc1/naapam.control.feather")

In [ ]:
from src import annote

df_control = annote.common.is_wt(df_control)
df_control = annote.control.count_tot(df_control)
df_control = annote.control.count_wt(df_control)
for tem in range(1, 5):
    df_control = annote.control.count_temN(df_control, tem)
df_control = annote.control.mutant_legal(df_control, max_m_synerr=0.05)
df_control = annote.control.barcode_legal(
    df_control,
    max_b_synerr=0.1,
    max_b_temNerr={1: 0.01, 2: 0.01, 3: 0.01, 4: 0.01},
)

In [ ]:
from src import collect

df_treat = collect.read_feather("/home/ljw/sdc1/naapam.treat.feather")

In [ ]:
from src import annote

df_treat = annote.common.is_wt(df_treat)
df_treat = annote.treat.count_tot(df_treat)
df_treat = annote.treat.count_wt(df_treat)
df_treat = annote.treat.mutant_legal(
    df_treat,
    max_up_del_size=20,
    max_down_del_size=20,
    max_rand_ins_size=10,
)
df_treat = annote.treat.barcode_legal(df_treat, min_count_tot=30)

In [ ]:
from src import mix

df_treat = mix.duplicate_treat(df_treat)
df_control = mix.duplicate_control(df_control)
df_treat = mix.merge_mutant(df_treat, df_control)
df_treat = mix.merge_barcode(df_treat, df_control)
df_treat.to_feather("/home/ljw/sdc1/naapam.mix.feather")

In [ ]:
from src import collect, annote

df_treat = collect.read_feather("/home/ljw/sdc1/naapam.mix.feather")
df_treat = annote.union.frequency_tot(df_treat)
df_treat = annote.union.frequency_nowt(df_treat)
df_treat = annote.union.frequency_tot_kim(df_treat)
df_treat = annote.union.frequency_nowt_kim(df_treat)
df_treat = annote.union.frequency_tot_adj(df_treat)
df_treat = annote.union.frequency_nowt_adj(df_treat)

In [ ]:
targets = ["frequency_nowt", "frequency_nowt_kim", "frequency_nowt_adj"]
df_treat = df_treat.query("barcode_legal and barcode_legal_ctl").drop(columns=["barcode_legal", "barcode_legal_ctl"])
df_treat.loc[~df_treat["mutant_legal"] | ~df_treat["mutant_legal_ctl"], targets] = float("nan")
df_treat = df_treat.drop(columns=["mutant_legal", "mutant_legal_ctl"])

In [ ]:
from src import draw

for tem in range(1,5):
    draw.over_up_del_size_on_tem(df_treat, tem, targets, aggfunc="sum")